In [2]:
import pickle 

In [6]:
import os

In [7]:
os.listdir()

['Imputation.ipynb',
 'RiskFactors.ipynb',
 'LinkPrediction.ipynb',
 '__init__.py',
 'EntityResolution.ipynb',
 'LinkPrediction-OntologyMatch.ipynb',
 'NodePrediction.ipynb',
 'Reasoner.ipynb']

In [3]:
data_path   = '../data/m2skg/'
filename    = 'assertion_train.pkl'

In [4]:
with open(data_path+filename,'rb') as f:
    data = pickle.load(f)

In [9]:
from ..src.gcn import GCN

ImportError: attempted relative import with no known parent package